In [10]:
import dolphin_memory_engine as dme
import memory_engine as me
import time

frame_rate = 60


# var to update with current frame/input
active_curve = None
active_frame = None

# store inputs
input_list = []
last_input_list = []
previous_curve_control = None
# used when I didn't want to save in real time - wrote to file and saved it there, but should be equally doable from input_list
# append values to list
with open('test.txt', 'r') as p_file:
    vals = p_file.readlines()
    for v in vals:
        input_list.append(float(v))

# ignore
# test = []
# hook into dme
dme.hook()

# print(dir(me))
while True:
    start_time = time.time()
    # assign variables - this measures current curve control input for frame
    curve_control = me.read_float(0x80890A24)
    if curve_control != previous_curve_control:
        print(curve_control)
        previous_curve_control = curve_control
        input_list.append(curve_control)
        print(input_list)
    # pitch frames - countdown of pitch, pitch = total duration of pitch / check for pitching or not
    active_pitch_frame = me.read_half_word(0x80890AF2)
    pitch_duration = me.read_half_word(0x80890AF4)

    # if a pitch is actively occurring
    if active_pitch_frame != active_frame:
        if last_input_list:
            if active_pitch_frame == pitch_duration:
                print(pitch_duration)
            # write float from list to current frame of pitch (len used to evaluate total length of pitch / what index of list to pull from)
            pitch_rewrite = me.write_float(0x80890A24, input_list[len(input_list) - 1 - active_pitch_frame])

            # update active_frame
            active_frame = active_pitch_frame

        # ignore
        # test.append(pitch_rewrite)
    if me.read_byte(0x80890AFE) == 4 and me.read_byte(0x80890AFE) != 1:
        if input_list:
            print(me.read_byte(0x80890AFE))
            print("pitch complete")
            last_input_list = input_list
            input_list = []
    frame_time = time.time() - start_time
    delay = max(1 / frame_rate - frame_time, 0)
    time.sleep(delay)




    #
    #     if curve_control != active_frame:
    #         print(curve_control)
    #         input_list.append(curve_control)
    #         active_curve = curve_control
    #         print(input_list)
    #
    #     if pitch_frames == 0:
    #         with open('test.txt', 'w') as f:
    #             for i in input_list:
    #                 f.write(str(i) + "\n")




0.0
[0.0, -0.000536529638338834, -0.001073059276677668, -0.001609588973224163, -0.002146118553355336, -0.002373737283051014, -0.002373737283051014, -0.002373737283051014, -0.002373737283051014, -0.002373737283051014, -0.002373737283051014, -0.0018372077029198408, -0.0007641484844498336, -0.0007641484844498336, 0.00030891079222783446, 0.0008454404305666685, 0.0013819700106978416, 0.0013819700106978416, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.002373737283051014, 0.0]
-0.000813953520264

IndexError: list index out of range

In [4]:
# import csv
# import time
# import dolphin_memory_engine as dme
# 
# csv_file_name = 'pitch_data.csv'
# 
# headers = ['pitch_id', 'strikes', 'balls', 'outs', 'pitcher', 'batter', 'pitch_duration', 'pitch_result', 'control_sequence', 'interrupted']
# 
# # Check if the CSV file exists and initialize it if it doesn't
# try:
#     with open(csv_file_name, 'x', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(headers)
# except FileExistsError:
#     pass  # File already exists, no need to initialize


In [19]:
import dolphin_memory_engine as dme
import memory_engine as me
import sys
import time
import csv

csv_filename = 'pitch_data.csv'
headers = ['pitch_id', 'pitch_duration', 'pitcher_id', 'pitch_type', 'curve_control']

# init csv file
try:
    with open(csv_filename, 'x', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
except FileExistsError:
    pass



is_mario_baseball = False
# hex equivalent to mssb filestring
# game_id = me.read_word(0x80000000)
# if game_id == 0x47595145:
#     is_mario_baseball = True
# else:
#     sys.exit(1)
    
frame_rate = 60
frame_count = 0
pitch_id = 0
event_id = 0
last_pitch_frame = -1
pitch_started = False
pitch_duration = 0


# halfword
batter_char_id = me.read_half_word(0x80890972)
pitcher_char_id = me.read_half_word(0x80890ADA)
# byte
batter_hand = me.read_byte(0x8089098B)
# byte
chem_links_on_base = me.read_byte(0x808909BA)
# I believe these are the locations of the bat 
bat_pos_x = me.read_float(0x8089095C)
bat_pos_z = me.read_float(0x80890964)
# byte - 0 is slap/star, 1/2 is charge
charge_status = me.read_byte(0x808909AD)
# byte
trimmed_bat = me.read_byte(0x8089099A)
# byte - 0 = curve, 1= charge, 2 = changeup, ff = default
pitch_type = me.read_byte(0x80890b21)

# batter swing and miss, batter sour, final result


# dme.hook()

previous_curve_control = None

active_pitch_frame = me.read_half_word(0x80890AF2)
pitch_duration = me.read_half_word(0x80890AF4)

while True:
    try:
        frame_count += 1
        start_time = time.time()
        
        new_active_pitch_frame = me.read_half_word(0x80890AF2)
        
        # Check if a new pitch has started
        if not pitch_started and new_active_pitch_frame != -1:
            pitch_started = True
            pitch_id += 1  # Increment pitch ID for a new pitch
            pitch_duration = new_active_pitch_frame
        
        # If a pitch is active
        if pitch_started:
            curve_control = me.read_float(0x80890A24)
            pitch_type = me.read_byte(0x80890b21)
            
            # Log at the start or during the pitch
            if new_active_pitch_frame != last_pitch_frame:
                with open(csv_filename, 'a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([pitch_id, pitch_duration, "Pitcher_ID", pitch_type, curve_control])
        
        # Detect end of pitch or interruption
        if pitch_started and (new_active_pitch_frame == -1 or new_active_pitch_frame > last_pitch_frame):
            pitch_started = False  # Reset for next pitch
        
        last_pitch_frame = new_active_pitch_frame
        
        # Frame rate control
        frame_time = time.time() - start_time
        delay = max(1 / frame_rate - frame_time, 0)
        time.sleep(delay)
    except RuntimeError:
        print("RuntimeError. The game has likely crashed.")
        break



KeyboardInterrupt: 

Pitch complete (30f) curve control values: [0.0, 0.0, 0.0004796511784661561, 0.0009593023569323123, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228, 0.0012499999720603228]
Pitch complete (41f) curve control values: [0.0, 0.0, -0.0008139535202644765, -0.001627907040528953, -0.0021212121937423944, -0.0021212121937423944, -0.0021212121937423944, -0.0021212121937423944, -0.0021212121937423944, -0.0021212121937423944, -0.0021212121937423944, -0.001307258615270257, -0.0004933

RuntimeError: Could not read memory at 2156464882

In [1]:
import csv
import os

# Function to append data to a CSV file with a header
def append_to_csv_with_header(file_path, pitcher_id, input_list):
    header = ['pitcher_id', 'curve_control_values']
    file_exists = os.path.isfile(file_path) and os.path.getsize(file_path) > 0
    
    with open(file_path, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if not file_exists:
            writer.writerow(header)
        curve_control_values_str = '+'.join(map(str, input_list))
        row = [pitcher_id, curve_control_values_str]
        writer.writerow(row)


In [2]:
import random

def load_data(file_path):
    data = {}
    try:
        with open(file_path, newline='') as csvfile:
            reader = csv.reader(csvfile)
            next(reader, None)  # Skip header row
            for row in reader:
                pitcher_id = row[0]
                curve_control_values = row[1].split('+')
                if str(pitcher_id) not in data:
                    data[str(pitcher_id)] = []
                data[str(pitcher_id)].append(curve_control_values)
    except FileNotFoundError:
        print("File not found. Please make sure the file path is correct.")
    return data


def choose_random_pitch(data, pitcher_id):
    if str(pitcher_id) in data:
        return random.choice(data[str(pitcher_id)])
    else:
        print(f"No data found for pitcher ID {pitcher_id}.")
        return None
        
    

In [3]:
def select_random_values(data, pitcher_id):
    if str(pitcher_id) in data:
        return random.choice(data[str(pitcher_id)])
    else:
        return None

In [6]:
import dolphin_memory_engine as dme
import memory_engine as me
import time

frame_rate = 60
# dme.hook()
file_path = 'pitch_data.csv'

data = load_data(file_path)

# Initialize variables
input_list = []
pitch_state = "waiting"  # waiting/active/ended

while True:
    start_time = time.time()
    
    # count down from pitch duration to 0. Is 0 when waiting
    active_pitch_frame = me.read_half_word(0x80890AF2)
    # -1 when not pitching. When pitch is initiated, displays total frame count of the pitch
    pitch_duration = me.read_half_word(0x80890AF4)
    curve_control = me.read_float(0x80890A24)
    pitcher_char_id = me.read_half_word(0x80890ADA)
    pitch_type = me.read_byte(0x80890b21)
    state = me.read_byte(0x80890AFE)
    mound_pos = me.read_float(0x80890a4c)
    # need to handle when the ball is actually hit
    # Detect the start of a pitch
    if pitch_duration != -1 and pitch_state == "waiting" and pitch_duration == active_pitch_frame:
        pitch_state = "active"
        pitch_index = 0
        print(mound_pos)
        random_pitch = choose_random_pitch(data, pitcher_char_id)

        input_list = []  # Reset the list for the new pitch
        
    # If a pitch is active, append curve control values
    if pitch_state == "active":
        if random_pitch is not None:
            me.write_float(0x80890A24, float(random_pitch[pitch_index]))
        input_list.append(curve_control)
        pitch_index += 1
        if active_pitch_frame == 0:
            pitch_state = "ended"
    
    # Handle the end of a pitch
    if pitch_state == "ended":
        print(f"Pitch complete ({len(input_list)}f) curve control values:", input_list)
        # save input_list to a file here
        # append_to_csv_with_header(file_path, pitcher_char_id, input_list)
        

        
        pitch_state = "waiting"  # Reset state to wait for the next pitch
        input_list = []  # Prepare for the next pitch
    
    # Handle the case where pitch_duration resets to -1, indicating no active pitch
    if pitch_duration == -1 and pitch_state != "waiting" and state == 4:
        pitch_state = "waiting"
    
    # Frame rate control to avoid overloading the CPU
    frame_time = time.time() - start_time
    delay = max(1 / frame_rate - frame_time, 0)
    time.sleep(delay)


-0.4000000059604645
Pitch complete (41f) curve control values: [0.0, 0.0, 0.000536529638338834, 0.00107305939309299, 0.001609588973224163, 0.002373737283051014, 0.0018372077029198408, 0.0018372077029198408, 0.0013006781227886677, 0.0007641484844498336, 0.00022761884611099958, 0.0, 0.0, 0.0, 0.0, -0.000536529638338834, -0.00107305939309299, -0.00107305939309299, -0.0018372077029198408, -0.0018372077029198408, -0.0018372077029198408, -0.0018372077029198408, -0.0018372077029198408, -0.0018372077029198408, -0.0018372077029198408, -0.0018372077029198408, -0.0018372077029198408, -0.0018372077029198408, -0.0013006781227886677, -0.0013006781227886677, -0.0007641484844498336, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000536529638338834]
-0.4000000059604645
Pitch complete (41f) curve control values: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.000536529638338834, -0.00107305939309299, -0.001609588973224163, -0.0018372077029198408, -0.0018372077029198408, -0.0018372077029198408, -0.0018372077

IndexError: list index out of range